# Home page

In [ ]:
import streamlit as st
import base64  # For embedding the WhatsApp logo

# Introduction Text
st.markdown("""
### Hello mate!\n
This app will make your buisness go with a boom!\n
        This app is a customizable AI-powered customer support chatbot. Features include:\n
            - Multilingual support
            - Booking management
            - Product information retrieval
            - Contextual query handling
            - Customizable responses for businesses
            - Financial Adivisor with Dept Analysis
""")
    

# Description
st.markdown("**Description about app's functions**")

with st.expander("About Query..."):
    st.markdown("""
    In **Query**, you can resolve any general queries or questions related to the buisness you have.\n
    """)

with st.expander("About Booking..."):
    st.markdown("""
In **Booking** page, you can book, cancel or reschedule any product or service you want directly.\n                          
""")
    
with st.expander("About Product Information..."):
    st.markdown("""
In **Product Information** page, you can get product information you want to search for.\n                                               
""")

with st.expander("About Customized Responses..."):
    st.markdown("""
In **Customized Responses** page, you can get all the detailed description of a product or service.\n                          
""")
    
with st.expander("About Chat Bot..."):
    st.markdown("""
In **Chat Bot** page, you can chat with our chatbot and access all the features mentioned above with chatting.\n
Like you can ask for a haircut, choose the right haircut offered by the salon, and book an appointment with it.\n                                               
""") 

with st.expander("About Financial Advisor..."):
    st.markdown("""
In **Financial Advisor** page, generates customized financial recommendations and visualizes financial distribution using graphs.\n
 The inputs include revenue, expenses, savings goals, and investment plans.\n                          
""")  

# Define the WhatsApp number (including country code) and the default message
WHATSAPP_NUMBER = "+916354252779"  # Replace with the actual WhatsApp number
DEFAULT_MESSAGE = "Hello! I would like to inquire about your services."  # Replace with your desired message

# URL to open WhatsApp chat
WHATSAPP_URL = f"https://wa.me/{WHATSAPP_NUMBER}?text={DEFAULT_MESSAGE}"

################################### Streamlit Page #######################################
st.title("Contact Us")

# Try using the full file path to avoid issues
logo_path = "whatsapp_logo.png"  # Path to the logo file (use full path if needed)

try:
    st.markdown(
        f"""
        <a href="{WHATSAPP_URL}" target="_blank">
            <img src="{logo_path}" alt="WhatsApp Logo" width="100">
        </a>
        """,
        unsafe_allow_html=True
    )
except Exception as e:
    st.error("WhatsApp logo could not be loaded. Please ensure the image file exists.")
    st.write(f"Error: {e}")

# Query page

In [ ]:
#Requirements
from langchain_groq import ChatGroq
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from googletrans import Translator  # Multilingual support


#LLM Model
llama = ChatGroq(
    model="LLaMA3-70B-8192",
    groq_api_key='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',
    temperature=0.0
)

translator = Translator()  # Translator for multilingual support

##################################### Base Chains ###########################################
# 1. Input Translation Chain
def translate_input(user_input):
    """Auto-detect and translate the user input to English."""
    try:
        detected_lang = translator.detect_language(user_input).result
        if detected_lang != "English":
            user_input = translator.translate(user_input, destination_language="English").result
        return user_input, detected_lang
    except Exception as e:
        #st.error(f"Error in language detection or translation: {e}")
        return user_input, "English"  # Default to English if detection fails

# 2. Output Translation Chain
def translate_output(response, target_lang):
    """Translate response back to the user's language."""
    try:
        if target_lang != "English":
            response = translator.translate(response, destination_language=target_lang).result
        return response
    except Exception as e:
        #st.error(f"Error in translating response: {e}")
        return response  # Return the original response if translation fails

# 3. Fallback Error Handling Chain
fallback_chain = (
    ChatPromptTemplate.from_template("""
You are a fallback assistant. If the system cannot handle the query, respond politely and suggest contacting support.
Input Query: "{query}"
""")
    | llama
    | StrOutputParser()
)

##################################### Advanced Chains ###########################################
# 4. General Query Chain
query_chain = (
    ChatPromptTemplate.from_template("""
You are a customer support assistant. The user has the query: "{query}". 
Respond with a clear and concise answer. If you don't know the answer, ask for more details or suggest contacting support.
""")
    | llama
    | StrOutputParser()
)

# 8. Contextual Query Chain
context_chain = (
    ChatPromptTemplate.from_template("""
You are a follow-up assistant. Based on the previous interaction: "{previous_interaction}", suggest a related query or provide further assistance.
""")
    | llama
    | StrOutputParser()
)

user_input = st.text_input("Ask your question here:")

if user_input:
    # Base Chain: Translate input to English
    translated_input, user_lang = translate_input(user_input)
    response = query_chain.invoke({"query": translated_input})
    # Base Chain: Translate output back to user language
    if response:
        translated_response = translate_output(response, user_lang)
        st.write("Response:", translated_response)

    # Contextual Suggestions
    if st.checkbox("Need more help? Get suggestions."):
        contextual_response = context_chain.invoke({"previous_interaction": translated_input})
        st.write("Suggestions:", contextual_response)

# Product information page

In [ ]:
#Requirements
from langchain_groq import ChatGroq
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from googletrans import Translator  # Multilingual support
import matplotlib.pyplot as plt  # For visualization
import random  # For generating dummy demand data

# LLM Model
llama = ChatGroq(
    model="LLaMA3-70B-8192",
    groq_api_key='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',
    temperature=0.0
)

translator = Translator()  # Translator for multilingual support

##################################### Base Chains ###########################################
# 1. Input Translation Chain
def translate_input(user_input):
    """Auto-detect and translate the user input to English."""
    try:
        detected_lang = translator.detect_language(user_input).result
        if detected_lang != "English":
            user_input = translator.translate(user_input, destination_language="English").result
        return user_input, detected_lang
    except Exception as e:
        return user_input, "English"  # Default to English if detection fails

# 2. Output Translation Chain
def translate_output(response, target_lang):
    """Translate response back to the user's language."""
    try:
        if target_lang != "English":
            response = translator.translate(response, destination_language=target_lang).result
        return response
    except Exception as e:
        return response  # Return the original response if translation fails

# 3. Fallback Error Handling Chain
fallback_chain = (
    ChatPromptTemplate.from_template("""
You are a fallback assistant. If the system cannot handle the query, respond politely and suggest contacting support.
Input Query: "{query}"
""")
    | llama
    | StrOutputParser()
)

##################################### Advanced Chains ###########################################
# 4. General Query Chain
# 6. Product Information Chain
product_info_chain = (
    ChatPromptTemplate.from_template("""
You are a knowledgeable sales assistant. The user is asking about: "{product}". 
Provide detailed information including features, pricing, and availability.
""")
    | llama
    | StrOutputParser()
)

# 8. Contextual Query Chain
context_chain = (
    ChatPromptTemplate.from_template("""
You are a follow-up assistant. Based on the previous interaction: "{previous_interaction}", suggest a related query or provide further assistance.
""")
    | llama
    | StrOutputParser()
)

##################################### Upselling Chain ###########################################
# 9. Upselling Chain
upselling_chain = (
    ChatPromptTemplate.from_template("""
You are an AI sales assistant. The user is interested in "{product}". 
Suggest related or higher-tier products/services and their potential benefits.
""")
    | llama
    | StrOutputParser()
)

##################################### Streamlit Application ###########################################
product = st.text_input("Product Name:")
if product:
    # Base Chain: Translate input to English
    translated_input, user_lang = translate_input(product)

    # Product Info Chain
    if product:
        response = product_info_chain.invoke({"product": product})
    else:
        response = fallback_chain.invoke({"query": "Missing product details."})
    
    # Base Chain: Translate output back to user language
    if response:
        translated_response = translate_output(response, user_lang)
        st.write("Response:", translated_response)

    # Contextual Suggestions
    if st.checkbox("Need more help? Get suggestions."):
        contextual_response = context_chain.invoke({"previous_interaction": translated_input})
        st.write("Suggestions:", contextual_response)

    # Upselling Recommendations
    st.subheader("Upselling Recommendations")
    upselling_response = upselling_chain.invoke({"product": product})
    st.write("Upselling Suggestions:", upselling_response)

    # Visualize Upselling Suggestions
    upsell_products = [f"{product} Pro", f"{product} Premium", f"{product} Bundle"]
    upsell_demand = [random.randint(20, 100) for _ in upsell_products]

    st.write("**Demand for Upselling Options:**")
    fig, ax = plt.subplots()
    ax.bar(upsell_products, upsell_demand, color="lightblue")
    ax.set_title("Upselling Demand Visualization")
    ax.set_xlabel("Products")
    ax.set_ylabel("Demand Score")
    st.pyplot(fig)


# Financial support page

In [ ]:
from langchain_groq import ChatGroq
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from googletrans import Translator  # Multilingual support
import matplotlib.pyplot as plt
import numpy as np

# LLM Model
llama = ChatGroq(
    model="LLaMA3-70B-8192",
    groq_api_key='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',
    temperature=0.0
)

translator = Translator()  # Translator for multilingual support

##################################### Base Chains ###########################################
# 1. Input Translation Chain
def translate_input(user_input):
    """Auto-detect and translate the user input to English."""
    try:
        detected_lang = translator.detect_language(user_input).result
        if detected_lang != "English":
            user_input = translator.translate(user_input, destination_language="English").result
        return user_input, detected_lang
    except Exception as e:
        return user_input, "English"  # Default to English if detection fails

# 2. Output Translation Chain
def translate_output(response, target_lang):
    """Translate response back to the user's language."""
    try:
        if target_lang != "English":
            response = translator.translate(response, destination_language=target_lang).result
        return response
    except Exception as e:
        return response  # Return the original response if translation fails

# 3. Fallback Error Handling Chain
fallback_chain = (
    ChatPromptTemplate.from_template("""
You are a fallback assistant. If the system cannot handle the query, respond politely and suggest contacting support.
Input Query: "{query}"
""")
    | llama
    | StrOutputParser()
)

##################################### Financial Advisor Chains ###########################################
# Custom Financial Advice Chain
financial_chain = (
    ChatPromptTemplate.from_template("""
You are a financial advisor for small businesses. The business owner provided the following details:
Revenue: {revenue}
Expenses: {expenses}
Savings Goal: {savings_goal}
Investment Plan: {investment_plan}
Debt: {debt}

Provide detailed financial recommendations with the following:
1. Ways to optimize expenses.
2. Strategies to achieve the savings goal.
3. Insights on the investment plan, including feasibility and potential risks.
4. Debt reduction strategies tailored to the user's financial condition.
5. Suggestions for reinvestment into the business.

Respond in a professional tone with actionable insights.
""")
    | llama
    | StrOutputParser()
)

##################################### Streamlit Financial Advisor Page ###########################################

# Form for user input
with st.form("financial_advisor_form"):
    st.header("Enter Business Financial Details:")
    
    # Financial inputs
    revenue = st.number_input("Monthly Revenue ($)", min_value=0, step=1)
    expenses = st.number_input("Monthly Expenses ($)", min_value=0, step=1)
    savings_goal = st.number_input("Savings Goal ($)", min_value=0, step=1)
    
    # Debt information
    debt = st.number_input("Outstanding Debt ($)", min_value=0, step=1)
    debt_interest = st.number_input(
        "Debt Interest Rate (%)", min_value=0.0, step=0.1,
        help="Enter the annual interest rate on your outstanding debt."
    )
    monthly_payment = st.number_input(
        "Monthly Debt Payment ($)", min_value=0, step=100,
        help="Enter the amount you pay monthly to service the debt."
    )
    
    # Investment plan section (before the button)
    st.subheader("Describe Your Investment Plan:")
    investment_plan = st.text_area(
        "Investment Plan (e.g., expand operations, invest in marketing, etc.)",
        placeholder="Briefly describe your investment plan"
    )

    # Submit button
    submit_financials = st.form_submit_button("Get Financial Advice")

    if submit_financials:
        if investment_plan and revenue > 0 and expenses > 0:
            # Generate Financial Advice
            response = financial_chain.invoke({
                "revenue": revenue,
                "expenses": expenses,
                "savings_goal": savings_goal,
                "investment_plan": investment_plan,
                "debt": f"{debt} at {debt_interest}% interest with ${monthly_payment}/month payment"
            })

            # Display Financial Advice
            st.subheader("Customized Financial Recommendations:")
            st.write(response)

            # Financial Distribution Visualization
            st.subheader("Financial Distribution:")
            financial_data = {
                "Revenue": revenue,
                "Expenses": expenses,
                "Savings Goal": savings_goal,
                "Debt Payment": monthly_payment * 12,
                "Remaining Funds": max(revenue - expenses - savings_goal - (monthly_payment * 12), 0)
            }

            fig, ax = plt.subplots(figsize=(6, 6))
            labels = financial_data.keys()
            sizes = financial_data.values()
            explode = (0.1, 0, 0, 0, 0)  # Emphasize 'Revenue'
            colors = ['#2E91E5', '#E15F99', '#1CA71C', '#FB0D0D', '#FFC300']

            ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors)
            ax.axis('equal')  # Equal aspect ratio ensures the pie is drawn as a circle.

            st.pyplot(fig)

            # Debt Analysis
            st.subheader("Debt Analysis:")
            total_debt_payment = monthly_payment * 12
            remaining_debt = debt - total_debt_payment if debt > total_debt_payment else 0
            interest_paid = debt * (debt_interest / 100)

            st.write(f"Total Debt Paid This Year: **${total_debt_payment:,.2f}**")
            st.write(f"Remaining Debt After a Year: **${remaining_debt:,.2f}**")
            st.write(f"Estimated Interest Paid This Year: **${interest_paid:,.2f}**")

            if remaining_debt > 0:
                st.write(
                    "Consider increasing your monthly debt payments or restructuring the debt to reduce interest payments."
                )
            else:
                st.write("Congratulations! You are on track to clear your debt.")
        else:
            st.warning("Please enter a valid investment plan to proceed.")


# Customisation page

In [ ]:
#Requirements
from langchain_groq import ChatGroq
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from googletrans import Translator  # Multilingual support

# LLM Model
llama = ChatGroq(
    model="LLaMA3-70B-8192",
    groq_api_key='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',
    temperature=0.0
)

translator = Translator()  # Translator for multilingual support

##################################### Base Chains ###########################################
# 1. Input Translation Chain
def translate_input(user_input):
    """Auto-detect and translate the user input to English."""
    try:
        detected_lang = translator.detect_language(user_input).result
        if detected_lang != "English":
            user_input = translator.translate(user_input, destination_language="English").result
        return user_input, detected_lang
    except Exception as e:
        #st.error(f"Error in language detection or translation: {e}")
        return user_input, "English"  # Default to English if detection fails

# 2. Output Translation Chain
def translate_output(response, target_lang):
    """Translate response back to the user's language."""
    try:
        if target_lang != "English":
            response = translator.translate(response, destination_language=target_lang).result
        return response
    except Exception as e:
        #st.error(f"Error in translating response: {e}")
        return response  # Return the original response if translation fails

# 3. Fallback Error Handling Chain
fallback_chain = (
    ChatPromptTemplate.from_template("""
You are a fallback assistant. If the system cannot handle the query, respond politely and suggest contacting support.
Input Query: "{query}"
""")
    | llama
    | StrOutputParser()
)

##################################### Advanced Chains ###########################################
# 4. Customization Chain
custom_chain = (
    ChatPromptTemplate.from_template("""
You are assisting a small business that specializes in {business_type}. Tailor your response to reflect the company's services and brand tone.
Give answer in long form.
The user query is: "{query}".
""")
    | llama
    | StrOutputParser()
)

# 8. Contextual Query Chain
context_chain = (
    ChatPromptTemplate.from_template("""
You are a follow-up assistant. Based on the previous interaction: "{previous_interaction}", suggest a related query or provide further assistance.
""")
    | llama
    | StrOutputParser()
)

# Create form to collect both inputs
with st.form("user_query_form"):
    business_type = st.text_input("Business Type (e.g., Restaurant, Salon, etc.):")
    user_input = st.text_input("Ask your question here:")

    # Submit button for the form
    submit_button = st.form_submit_button(label="Submit")

if submit_button:
    if business_type and user_input:
        # Translate input to English
        translated_input, user_lang = translate_input(user_input)

        # Generate response using the custom chain
        response = custom_chain.invoke({"business_type": business_type, "query": translated_input})
        
        # Translate output back to user language
        if response:
            translated_response = translate_output(response, user_lang)
            st.write("Response:", translated_response)

        # Contextual Suggestions
        if st.checkbox("Need more help? Get suggestions."):
            contextual_response = context_chain.invoke({"previous_interaction": translated_input})
            st.write("Suggestions:", contextual_response)
    else:
        st.warning("Please provide both Business Type and your Question.")


# Booking page

In [ ]:
#Requirements
from langchain_groq import ChatGroq
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from googletrans import Translator  # Multilingual support


#LLM Model
llama = ChatGroq(
    model="LLaMA3-70B-8192",
    groq_api_key='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',
    temperature=0.0
)

translator = Translator()  # Translator for multilingual support

##################################### Base Chains ###########################################
# 1. Input Translation Chain
def translate_input(user_input):
    """Auto-detect and translate the user input to English."""
    try:
        detected_lang = translator.detect_language(user_input).result
        if detected_lang != "English":
            user_input = translator.translate(user_input, destination_language="English").result
        return user_input, detected_lang
    except Exception as e:
        #st.error(f"Error in language detection or translation: {e}")
        return user_input, "English"  # Default to English if detection fails

# 2. Output Translation Chain
def translate_output(response, target_lang):
    """Translate response back to the user's language."""
    try:
        if target_lang != "English":
            response = translator.translate(response, destination_language=target_lang).result
        return response
    except Exception as e:
        #st.error(f"Error in translating response: {e}")
        return response  # Return the original response if translation fails

# 3. Fallback Error Handling Chain
fallback_chain = (
    ChatPromptTemplate.from_template("""
You are a fallback assistant. If the system cannot handle the query, respond politely and suggest contacting support.
Input Query: "{query}"
""")
    | llama
    | StrOutputParser()
)

##################################### Advanced Chains ###########################################
# 5. Booking Management Chain
booking_chain = (
    ChatPromptTemplate.from_template("""
You are a virtual assistant that manages bookings for a business. The user wants to {action} for a service: "{service}" on the following date and time {date}{time}. 
If they provide a time or date, confirm the booking; if not, ask for more details.
""")
    | llama
    | StrOutputParser()
)

# 8. Contextual Query Chain
context_chain = (
    ChatPromptTemplate.from_template("""
You are a follow-up assistant. Based on the previous interaction: "{previous_interaction}", suggest a related query or provide further assistance.
""")
    | llama
    | StrOutputParser()
)

service = st.text_input("Service Type (e.g., Haircut, Meeting, etc.):")

action = st.text_input("Action (e.g., Book, Cancel, Reschedule):")

date = st.date_input("When to perfom action?", value=None)

time = st.time_input("Set an alarm for", value=None)


if service:
    # Base Chain: Translate input to English
    translated_input, user_lang = translate_input(service)
    if service and action:
        response = booking_chain.invoke({"action": action, "service": service, "date": date, "time": time})
    else:
        response = fallback_chain.invoke({"query": "Missing service or action details."})


    # Base Chain: Translate output back to user language
    if response:
        translated_response = translate_output(response, user_lang)
        st.write("Response:", translated_response)

    # Contextual Suggestions
    if st.checkbox("Need more help? Get suggestions."):
        contextual_response = context_chain.invoke({"previous_interaction": translated_input})
        st.write("Suggestions:", contextual_response)

# Chat Bot page

In [ ]:
#Requirements
from langchain_groq import ChatGroq
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from googletrans import Translator  # Multilingual support
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

#LLM Model
llama = ChatGroq(
    model="LLaMA3-70B-8192",
    groq_api_key='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',
    temperature=0.0
)

translator = Translator()  # Translator for multilingual support
##################################### Base Chains ###########################################

Chat_chain = (
    
    ChatPromptTemplate.from_template("""
You are an ai chatbot fine tuned for buisness and sales.
You are made for assistance for Small buisnesses to grow them. The customer will chat {userinput} with you and you will service them.
Here is the following services you will provide:
                                     resolve the customer queries; 
                                     provide product information; 
                                     book, cancel, or re-schedule any product or service; 
                                     provide customized solutions for queries; 
                                     Help customer to choose the best product or service according to their needs or requirements.
""")
    | llama
    | StrOutputParser()  
)


# 1. Input Translation Chain
def translate_input(userinput):
    """Auto-detect and translate the user input to English."""
    try:
        detected_lang = translator.detect_language(userinput).result
        if detected_lang != "English":
            userinput = translator.translate(userinput, destination_language="English").result
        return userinput, detected_lang
    except Exception as e:
        #st.error(f"Error in language detection or translation: {e}")
        return userinput, "English"  # Default to English if detection fails

# 2. Output Translation Chain
def translate_output(response, target_lang):
    """Translate response back to the user's language."""
    try:
        if target_lang != "English":
            response = translator.translate(response, destination_language=target_lang).result
        return response
    except Exception as e:
        #st.error(f"Error in translating response: {e}")
        return response  # Return the original response if translation fails


if "messages" not in st.session_state:
    st.session_state.messages = []

# getting User input
userinput = st.chat_input("How can I help you?")
with st.chat_message("user"):
        st.write(userinput)


if userinput:
    message = st.chat_message("assistant")
    #message.write(cbt_chain.invoke(user_input))
    # Base Chain: Translate input to English
    translated_input, user_lang = translate_input(userinput)
    st.session_state.messages.append({"role": "user", "content": userinput})
    response = Chat_chain.invoke(userinput)
    # Base Chain: Translate output back to user language
    if response:
        translated_response = translate_output(response, user_lang)
        st.write("Response:", translated_response)     
    st.session_state.messages.append({"role": "assistant", "content": response})
    
    
    
# Display chat history
for message in st.session_state.messages:
    if message["role"] == "user":
        st.write(f"You: {message['content']}")
    else:
        st.write(f"Bot: {message['content']}")

# Main page

In [ ]:
import streamlit as st

# Title
st.title("AI Customer Support Assistant")

pages = {
    "Home": [
        st.Page("Home_sb.py", title="Welcome!"),
    ],
    "General Query": [
        st.Page("Query.py", title="Query"),
    ],
    "Booking Management": [
        st.Page("Booking.py", title="Booking."),
    ],
    "Product Information": [
        st.Page("Product_info.py", title="Product Information"),
    ],
    "Customized Responses": [
        st.Page("Customization.py", title="Customized Responses"),
    ],
    "Chat Bot": [
        st.Page("Chatbot.py", title="Chat Bot"),
    ],
    "Financial Advisor": [
        st.Page("Financial.py", title="Financial Advisor"),
    ]
}

# Navigation page execution
pg = st.navigation(pages)
pg.run()